In [17]:
import cv2
import tensorflow as tf
from keras.applications.mobilenet import  MobileNet
from keras.models import Model
from keras.callbacks import EarlyStopping

In [4]:
def get_model():
    # Tạo base model
    base_model = MobileNet(include_top=False, weights="imagenet", input_shape=(224,224,3))
    # Tạo model chính
    x = base_model.output
    # Add some new Fully connected layers 
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    prediction_layer = tf.keras.layers.Dense(5, activation='softmax')
    outputs = prediction_layer(x)

    # Đóng băng các layer của base_model
    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.inputs, outputs= outputs)
    return model

model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [5]:
data_folder = "datasets"

train_datagen = ImageDataGenerator(preprocessing_function= keras.applications.mobilenet.preprocess_input,rotation_range=0.2,
                                   width_shift_range=0.2,   height_shift_range=0.2,shear_range=0.3,zoom_range=0.5,
                                   horizontal_flip=True, vertical_flip=True,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(data_folder,
                                                    target_size=(224, 224),
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(
    data_folder,  # same directory as training data
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation')  # set as validation data

classes = train_generator.class_indices
print(classes)
classes = list(classes.keys())

Found 13778 images belonging to 5 classes.
Found 3441 images belonging to 5 classes.
{'car': 0, 'cat': 1, 'dog': 2, 'flower': 3, 'people': 4}


In [14]:
n_epochs = 3
batch_size = 100
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

step_train = train_generator.n//batch_size
step_val = validation_generator.n//batch_size

model.fit_generator(generator=train_generator, steps_per_epoch=step_train,
                    validation_data=validation_generator,
                    validation_steps=step_val,
                    callbacks=[es],
                    epochs=n_epochs)

C:\Users\DELL\AppData\Local\Temp\ipykernel_12252\3981974021.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator, steps_per_epoch=step_train,


Epoch 1/3
137/137 [==============================] - 331s 2s/step - loss: 0.2376 - accuracy: 0.9118 - val_loss: 0.2047 - val_accuracy: 0.9269
Epoch 2/3
137/137 [==============================] - 360s 3s/step - loss: 0.2221 - accuracy: 0.9162 - val_loss: 0.2045 - val_accuracy: 0.9228
Epoch 2: early stopping


In [18]:
model.save('models/model_2/model.h5')

INFO:tensorflow:Assets written to: models/model_2\assets


INFO:tensorflow:Assets written to: models/model_2\assets


In [ ]:
# build model from h5 to json file

# script: tensorflowjs_converter --input_format=keras .\models\model_2\model.h5 .\models\model_2\build